## data

In [0]:
import matplotlib.pyplot as plt
import numpy as np

file_data   = "mnist.csv"
handle_file = open(file_data, "r")
data        = handle_file.readlines()
handle_file.close()

In [0]:
size_row    = 28    # height of the image
size_col    = 28    # width of the image

num_image   = len(data)
count       = 0     # count for the number of images

In [0]:
# normalize the values of the input data to be [0, 1]
def normalize(data):

    data_normalized = (data - min(data)) / (max(data) - min(data))

    return(data_normalized)

In [0]:
# example of distance function between two vectors x and y
def distance(x, y):

    d = (x - y) ** 2
    s = np.sum(d)
    # r = np.sqrt(s)

    return(s)

In [0]:
# make a matrix each column of which represents an images in a vector form
list_image  = np.empty((size_row * size_col, num_image), dtype=float)
list_label  = np.empty(num_image, dtype=int)

for line in data:
    line_data   = line.split(',')
    label       = line_data[0]
    im_vector   = np.asfarray(line_data[1:])
    im_vector   = normalize(im_vector) # 이미지 데이터 정규화 

    list_label[count]       = label
    list_image[:, count]    = im_vector

    count += 1

In [6]:
print(list_image.shape, list_label.T.shape) # 확인 

(784, 10000) (10000,)


In [7]:
list_image.T[0].shape # 이 형태가 10000가 있는 거라고 생각하면 된다. 

(784,)

In [8]:
len(list_image.T) #이미지 벡터의 수 

10000

In [9]:
len(list_label) # 라벨의 수 

10000

## optimization
- input으로 넣은 행렬끼리의 연산에 주의(행렬 shape에 주의하기)
- randomly initialize weights
- Implement foward propagation to get h(x) for any x
- Implement code to compute cost function J
- Implement backprop to compute partial derivations

- and training the neural network
- Use gradient descent with back propagation to minimize J(theta) as function of parameter theta.


In [0]:
# 먼저 theta의 행렬의 형태에 대한 정의가 필요하다.  
# 랜덤으로 정한 theta (U,V,W)에 해당 
theta1 = np.random.random((196, 785))
theta2 = np.random.random((49, 197))
theta3 = np.random.random((10, 50))

# 정한 theta들의 리스트
theta_vector = [theta1, theta2, theta3]

In [0]:
# h, sigmoid함수 정의 
# activation function으로 sigmoid를 사용할 예정에 있음 
def h(theta,X):
  return np.dot(X,theta)

def sigmoid(x):
  return np.exp(-np.maximum(x,0))/(1.0+np.exp(-np.abs(x))) #sigmoid함수 변형식 사용하여 정의함

# 미분한 sigmoid식
def Dsigmoid(x):
  return sigmoid(x)*(1-sigmoid(x))

In [0]:
list_image = np.insert(list_image, 0,1,axis=0)

In [13]:
list_image.shape # bias추가한 shape확인 

(785, 10000)

**순전파 + cost function**

In [0]:
# 순전파 함수 수정한 함수 
def Forward(image_vector,theta_vector):
    features = image_vector
    z_each_layer = []

    # theta의 수대로 반복하고, 그에따라 각각 저장해야 한다. 
    for i in range(len(theta_vector)):  
        theta = theta_vector[i]
        z = theta.dot(features).reshape((theta.shape[0],1))
        a = sigmoid(z)
        z_each_layer.append( (z, a) ) # z와 a값 모두 저장한다 

        if i == len(theta_vector)-1:
            return np.array(z_each_layer)

        a = np.insert(a,0,1) # bias 붙임 
        features = a # 다음 

In [0]:
h = Forward(list_image.T[0],theta_vector)
#print(h) #계산이 잘 됨을 확인

In [0]:
# Objective function에 대한 정의 
def ObjectiveFunction(thetha_vector, image_vector,label_list):
  Allcost = 0. # 전체 곱한 값을 저장할 변수
  m = len(list_label) # 전체 샘플의 수(image의 수)
  # train
  for i in range(m):
    new_feature = image_vector.T[i]
    h = Forward(new_feature.T,thetha_vector)[-1][1] # 순전파로 계산한 값의 h
    
    tmp = np.zeros((10,1))
    tmp[label_list[i]-1] = 1 #원핫 인코딩을 통해 원래 label을 지정함 

    # compute cost
    cost = -tmp.T.dot(np.log(h))-(1-tmp.T).dot(np.log(1-h))

    # 전체 cost는 각각의 cost를 합한값에 해당한다. 
    Allcost += cost
  
  # 전체 sample수로 나누는 항에 해당하는 부분 
  Allcost = Allcost/m

  return Allcost

In [17]:
# 랜덤으로 정한 theta에 대하여 cost값 계산해본다. (의미없음. 마찬가지로 계산이 되는지를 확인함)
ObjectiveFunction(theta_vector, list_image, list_label)

array([[9.69087995]])

**역전파**

In [0]:
# 각 layer에서의 노드수를 지정하여 변수로 저장 
input_layer = 785
first_hidden_layer = 196
second_hidden_layer = 49
output_layer = 10

In [0]:
# 순전파로 최종 classify된 label을 정하는 함수 
def NN(image_vector, theta_vector):
  outputLabel = range(1,11) # 1~10의 label을 가진 output임
  for i in range(10000):
    feature = image_vector.T[i]
    outputReal = Forward(feature,theta_vector)
    return outputLabel[np.argmax(outputReal[-1][1])]

In [0]:
# 정확도
def accuracy(image_vector, theta_vector, label_list):
  count = 0 # 맞은 class를 세는 변수
  all = image_vector.T.shape[0]
  for i in range(all):
    if(NN(image_vector[i],theta_vector)==label_list[i]): #맞은 개수
      count+=1
  return (count/all)*100 

In [0]:
# 역전파 함수  

# hyperparameter 
iteration = 10
alpha  = 0.01
m = len(list_label)

def Back(theta_vector, image_vector, label_list):

  # 델타값의 initialization
  D1 = np.zeros((first_hidden_layer, input_layer))          # (196, 785)
  D2 = np.zeros((second_hidden_layer,first_hidden_layer+1)) # (49, 197)
  D3 = np.zeros((output_layer, second_hidden_layer+1))      # (10,50)

  # update값 저장할 리스트 (이후 plotting위함)
  J_history = [] 
  acc_history = []

  # 전체 샘플수에 대하여 계산한다. 
  SampleLen = len(list_label)

  for j in range(iteration):  
   for i in range(SampleLen):

      tmp1 = theta_vector[0]
      tmp2 = theta_vector[1]
      tmp3 = theta_vector[2]

      # J_history, acc_history값 저장
      J_history.append(ObjectiveFunction(theta_vector, image_vector, label_list))
      #acc_history.append(accuracy(image_vector, theta_vector, label_list))

      # 편미분 term 계산 
      feature = image_vector.T[i] # 각 샘플마다의 image vector을 의미한다. 
      a1 = feature.reshape((input_layer,1))

     # 순전파의 결과로 나오는 (3,2)matrix의 값을 각각의 변수에 저장한다. 
      foward = Forward(feature, theta_vector)
      z2 = foward[0][0]
      a2 = foward[0][1]
      z3 = foward[1][0]
      a3 = foward[1][1]
      a4 = foward[2][0]
      z4 = foward[2][1]

      # 10개 레이블의 원핫 인코딩을 위한 더미 값 만들기
      tmp = np.zeros((10,1))
      tmp[label_list[i]-1] = 1
      delta4 = a4 - tmp # 가장 마지막에 있는 layer에서의 작은델타값
      delta3 = theta_vector[2].T[1:,:].dot(delta4)*Dsigmoid(z3)
      a3 = np.insert(a3, 0, 1, axis =0)
      delta2 = theta_vector[1].T[1:,:].dot(delta3)*Dsigmoid(z2)
      a2 = np.insert(a2, 0, 1, axis =0)
      # + delta1은 input 값에 대한 에러이므로 계산할 필요가 없다
    
      # big delta에 대한 계산
      D1 += delta2.dot(a1.T)
      D2 += delta3.dot(a2.T)
      D3 += delta4.dot(a3.T)

      # update theta
      tmp3 = theta_vector[2] - alpha*(D3/m)
      tmp2 = theta_vector[1] - alpha*(D2/m)
      tmp1 = theta_vector[0] - alpha*(D1/m)

  # 최종 theta1,2,3
  theta_vector[0] = tmp1
  theta_vector[1] = tmp2
  theta_vector[2] = tmp3 

  return theta_vector, J_history, acc_history

In [33]:
theta_vector, J_history, acc_history = Back(theta_vector, list_image, list_label)

ValueError: ignored

In [0]:
# cost값 plotting
plt.figure(figsize=(7, 5))
plt.plot(range(len(J_history)),J_history,'b')

In [0]:
# theta확인 
theta_vector

In [0]:
# accuracy plotting
plt.figure(figsize=(5, 5))
plt.plot(range(len(acc_history)),acc_history,'b')

In [0]:
# plot the accuracy value
acc_history[-1]

In [0]:
# plot the classification example
